# Piecewise Linear Revenue Optimization Model

[![Investment_project.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity_3.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity_3.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity_3.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity_3.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity_3.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Demand_elasticity/demand_elasticity_3.ipynb)](https://colab.ampl.com)

This model is designed for businesses seeking to determine optimal pricing and sales quantities under inventory and demand constraints. By leveraging piecewise linear pricing, it provides a flexible approach to maximize revenue while balancing supply and customer demand dynamics.

[*Partner with the AMPL team to transform complex problems into optimized solutions. AMPL consulting services combine deep technical knowledge with industry-leading insights, helping you unlock the full potential of optimization within your organization.*](https://ampl.com/services/)

Tags: Lessons, ampl, MIP, 

Notebook author: Mikhail Riabtsev <<mail@solverytic.com>>
***

# 1. Download Necessary Extensions and Libraries

In [9]:
# Install dependencies
%pip install -q amplpy pandas
import pandas as pd                 # Loading panda to work with pandas.DataFrame objects (https://pandas.pydata.org/)
import numpy as np                  # Loading numpy to perform multidimensional calculations numpy.matrix (https://numpy.org/)

Note: you may need to restart the kernel to use updated packages.


In [10]:
'''# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["cbc", "highs", "gurobi"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics'''

'# Google Colab & Kaggle integration\nfrom amplpy import AMPL, ampl_notebook\n\nampl = ampl_notebook(\n    modules=["cbc", "highs", "gurobi"],  # modules to install\n    license_uuid="default",  # license to use\n)  # instantiate AMPL object and register magics'

In [11]:
from amplpy import AMPL
ampl = AMPL()                       # create a new AMPL object with all default settings

# 2. Model description

This model optimizes revenue from the sale of a product using a piecewise linear price-demand function. The goal is to determine the optimal pricing strategy and corresponding quantity sold at each price step to maximize total revenue while adhering to constraints on available inventory and demand.

## Model Components

### Parameters

- $total\_Quantity$: The total quantity of the product available for sale (≥0).
- $unit\_Cost$: The unit cost of the product (≥0).
- $nStep$: The number of price steps in the piecewise price-demand function (≥1).
- $demand[i]$: The maximum demand for the product at the i-th price step (≥0).
- $price[i]$: The price corresponding to the i-th price step (≥0).

### Decision Variables

- $Quantity\_Sold[i]$: The quantity of the product sold at the i-th price step (≥0).
- $Price\_Selected[i]$: Binary variable indicating whether the i-th price step is selected (1 if selected, 0 otherwise).

### Objective Function

-  ***Maximize Total Revenue***: The model maximizes revenue by summing the product of the quantity sold at each step and the corresponding price adjusted for unit cost.

### Constraints
- ***Single Price Step Selection***:
Ensures that only one price step is selected:

   $\sum_{i=1}^{nStep} {Price\_Selected}[i] = 1$

- ***Total Quantity Constraint***:

   Ensures that the total quantity sold does not exceed the available inventory:

   $\sum_{i=1}^{nStep} {Quantity\_Sold}[i] \leq {total\_Quantity}$

- ***Price Step Demand Limit***:

   Restricts the quantity sold at each price step to the corresponding demand level if the price step is selected:

   $Quantity\_Sold[i] \leq {demand}[i] \cdot \text{Price\_Selected}[i], \quad \forall i \in \{1, \ldots, \text{nStep}\}$

# 3. AMPL Model Formulation

In [12]:
%%writefile demand_elasticity_3_model.mod
reset;
### PARAMETERS
param total_Quantity >= 0;              # Total quantity of products available for sale
param unit_Cost >= 0;                   # Unit cost of the product

param nStep integer >= 1;               # Number of steps in the piecewise linear price function
param demand {1..nStep+1} >= 0;         # Demand values at each step
param price {1..nStep+1} >= 0;          # Price for each step


### VARIABLES
var Quantity_Sold {1..nStep} >= 0;      # Quantity sold at each price step
var Price_Selected {1..nStep} binary;   # Binary variable indicating whether the price step is selected


### OBJECTIVE
maximize Total_Revenue:                 # Maximize total revenue (profit from sales)
    sum {i in 1..nStep} 
    <<demand[i]; {p in i..i+1} price[p] 
    - unit_Cost>> Quantity_Sold[i];     # Here we use a piecewise linear function for a limited range i..i+1


### CONSTRAINTS
s.t. Single_Price:                      # Only one price step can be selected
    sum {i in 1..nStep} Price_Selected[i] = 1;

s.t. Quantity_Limit:                    # The total quantity sold cannot exceed available quantity
    sum {i in 1..nStep} Quantity_Sold[i] <= total_Quantity;

s.t. Price_Upper_Bound {i in 1..nStep}: # Quantity sold cannot exceed upper price step
    Quantity_Sold[i] <= demand[i] * Price_Selected[i];

Overwriting demand_elasticity_3_model.mod


# 4. Load data

In [13]:
ampl.read('demand_elasticity_3_model.mod')                        # Load the AMPL model from the file

# Set global parameters for the model
ampl.param['total_Quantity'] = 600                                # Total available quantity for sale
ampl.param['unit_Cost'] = 2                                       # Unit cost of the product
ampl.param['nStep'] = 13                                          # Number of price steps

# Define the data points as (demand, price) pairs for each step
data = {
   1: (0, 10), 
   2: (1, 8), 
   3: (5, 7.5),
   4: (10, 7),
   5: (15, 6.5),
   6: (30, 6),
   7: (60, 5.5),
   8: (100, 5),
   9: (150, 4.5),
   10: (200, 4),
   11: (240, 3.5),
   12: (270, 3),
   13: (280, 2.5),
   14: (290, 2)}

# Set the demand and price parameters in AMPL
demand_param = ampl.getParameter("demand")                        # Get the 'demand' parameter
price_param = ampl.getParameter("price")                          # Get the 'price' parameter
for i, (demand_value, price_value) in data.items():               # Loop through data points
    demand_param.set(i, demand_value)                             # Assign demand value to each step
    price_param.set(i, price_value)                               # Assign price value to each step

# 5. Solve problem

In [14]:
# Set the solver type for use in solving the problems
solver = 'cplex'  # Use CBC solver for optimization tasks

ampl.option['show_stats'] = 0 # Show problem size statistics (default: 0)
ampl.option['display_1col'] = 0 # Disable single-column data display
#ampl.option['omit_zero_rows'] = 1 # Hide rows with zero values
#ampl.option['omit_zero_cols'] = 1 # Hide columns with zero values
ampl.option['mp_options'] = 'outlev=1 lim:time=20'   # Configure CBC options (output level and time limit)

ampl.solve(solver=solver, verbose=False)   # Solve the optimization problem using CBC solver  

## 6. Display results

In [15]:
# Display results for key variables
ampl.display('_varname', '_var', '_var.lb', '_var.ub', '_var.rc', '_var.slack')
ampl.display('_conname', '_con', '_con.body', '_con.lb', '_con.ub', '_con.slack')
ampl.display('_objname', '_obj')

:          _varname        _var _var.lb   _var.ub  _var.rc _var.slack    :=
1    'Quantity_Sold[1]'       0     0            0     0         0
2    'Quantity_Sold[2]'       0     0     Infinity    -6         0
3    'Quantity_Sold[3]'       0     0     Infinity    -5.5       0
4    'Quantity_Sold[4]'       0     0     Infinity    -5         0
5    'Quantity_Sold[5]'       0     0     Infinity    -4.5       0
6    'Quantity_Sold[6]'       0     0     Infinity    -4         0
7    'Quantity_Sold[7]'       0     0     Infinity    -3.5       0
8    'Quantity_Sold[8]'       0     0     Infinity    -3         0
9    'Quantity_Sold[9]'       0     0     Infinity    -2.5       0
10   'Quantity_Sold[10]'    200     0     Infinity    -2       200
11   'Quantity_Sold[11]'      0     0     Infinity    -1.5       0
12   'Quantity_Sold[12]'      0     0     Infinity    -1         0
13   'Quantity_Sold[13]'      0     0     Infinity    -0.5       0
14   'Price_Selected[1]'      0     0            1   

# 7. Retrieve solution in Python

In [16]:
# Initialize an empty dictionary to store AMPL variable data
amplvar = dict()

# Prepare a list of AMPL variables
list_of_ampl_variables = [item[0] for item in ampl.get_variables()]

# Iterate over each variable name in the list
for key_ampl in list_of_ampl_variables:
    # Skip certain variables that are not to be processed (these variables won't be included in the output)
    if key_ampl not in ['']:
        # Convert the AMPL variable data to a pandas DataFrame
        df = ampl.var[key_ampl].to_pandas()
        # Filter the DataFrame to include only rows where the variable's value is greater than a small threshold (1e-5)
        filtered_df = df[df[f"{key_ampl}.val"] > 1e-5]
        # Round the values in the DataFrame to two decimal places
        rounded_df = filtered_df.round(2)
        # Convert the filtered DataFrame to a dictionary and add it to the amplvar dictionary
        amplvar[key_ampl] = rounded_df #.to_dict(orient='records')
print (amplvar[key_ampl])

    Quantity_Sold.val
10                200
